<a href="https://colab.research.google.com/github/FLafage/Cours_Python/blob/main/Devoir_Test_duckdb_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Devoir de François pour tester duckdb en python**

In [ ]:
import pandas as pd
import geopandas as gpd
import duckdb

In [ ]:
url = "https://object.files.data.gouv.fr/data-pipeline-open/siren/stock/StockUniteLegale_utf8.parquet"

In [ ]:
duckdb.sql('SET enable_progress_bar = true;') # pour afficher la progress bar des requêtes

La première étape consiste à créer une vue pour éviter de réécrire chaque fois l'url

In [ ]:
duckdb.sql(f'CREATE OR REPLACE VIEW siren AS from read_parquet("{url}")')

La première requête : connaître le nombre de lignes (attention : le résultat est instantané)

In [ ]:
duckdb.sql('select count(*) from siren')

Attention, ici on cherche à décrire les variables de la table siren ! mais bon c'est encore raisonnable. C'est beaucoup plus long. La commande .df permet de convertir le résultat d'une requête duckcb en pandas.

In [ ]:
duckdb.sql("SUMMARIZE siren").df()

Maintenant, on va compter toutes les entreprises aquacoles de mer avec le code APE 03.21Z

In [ ]:
duckdb.sql('''
    SELECT count(*)
    FROM siren
    WHERE activitePrincipaleUniteLegale = '03.21Z'
''')

On va maintenant récupérer toutes les entreprises, c'est long, mais c'est possible (plus de 10 minutes).

In [ ]:
aqua_mer = duckdb.sql('''
    SELECT *
    FROM siren
    WHERE activitePrincipaleUniteLegale = '03.21Z'
    ''').df()

Les devoirs donnés par François sont les suivants :

*   Calculer le nombre d'entreprises par activitePrincipaleUniteLegale
*   Calculer le nombre d'entreprises par categorieEntreprise

Puis prendre le fichier des https://www.data.gouv.fr/datasets/geolocalisation-des-etablissements-du-repertoire-sirene-pour-les-etudes-statistiques/informations des etablissements et calculer par commune le nombre d'établissements. Puis faire la carte ! Le fond de carte est ici https://creacartes.s3.eu-west-3.amazonaws.com/2025/francemetro/commune_francemetro_2025.gpkg Vous pouvez directement le charger avec l'url...

Calcul du nombre d'entreprises par activité principale

In [ ]:
nombre_entreprises_par_activite_principale = duckdb.sql('''
    SELECT
        activitePrincipaleUniteLegale, count(*)
    FROM siren
    GROUP BY activitePrincipaleUniteLegale
''').show()

Calcul du nombre d'entreprises par categorieEntreprise

In [ ]:
nombre_entreprises_par_activite_entreprise = duckdb.sql('''
    SELECT
        categorieEntreprise, count(*)
    FROM siren
    GROUP BY categorieEntreprise
''').show()

Calcul du nombre de commune par établissement

In [ ]:
url_etablissement = "https://object.files.data.gouv.fr/data-pipeline-open/siren/geoloc/GeolocalisationEtablissement_Sirene_pour_etudes_statistiques_utf8.parquet"

In [ ]:
duckdb.sql(f'CREATE OR REPLACE VIEW SIRET AS from read_parquet("{url_etablissement}")')

In [ ]:
nombre_etablissements_par_commune = duckdb.sql('''
    SELECT
        PLG_CODE_COMMUNE, count(*)
    FROM SIRET
    GROUP BY PLG_CODE_COMMUNE
''').df()

nombre_etablissements_par_commune

Création de la carte :

In [ ]:
url_carte = "https://creacartes.s3.eu-west-3.amazonaws.com/2025/francemetro/commune_francemetro_2025.gpkg"

In [ ]:
carte = gpd.read_file(url_carte)

In [ ]:
carte.head()

In [ ]:
carte_nbr_entreprises_commune = carte.merge(nombre_etablissements_par_commune, left_on="code", right_on="plg_code_commune")

carte_nbr_entreprises_commune

In [ ]:
carte_nbr_entreprises_commune.plot(
    column='count_star()',
    legend=True
)